<a href="https://colab.research.google.com/github/rgvananth/Pytorch_ML/blob/master/BERT_Sentiment_Analysis_Parallel_TPU_Traning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installs PyTorch, PyTorch/XLA, and Torchvision
# Copy this cell into your own notebooks to use PyTorch on Cloud TPUs 
# Warning: this may take a couple minutes to run

import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3727  100  3727    0     0  41876      0 --:--:-- --:--:-- --:--:-- 41876
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200325 ...
Uninstalling torch-1.4.0:
Done updating TPU runtime: <Response [200]>
  Successfully uninstalled torch-1.4.0
Uninstalling torchvision-0.5.0:
  Successfully uninstalled torchvision-0.5.0
Copying gs://tpu-pytorch/wheels/torch-nightly+20200325-cp36-cp36m-linux_x86_64.whl...
\ [1 files][ 83.4 MiB/ 83.4 MiB]                                                
Operation completed over 1 objects/83.4 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200325-cp36-cp36m-linux_x86_64.whl...
- [1 files][114.5 MiB/114.5 MiB]                                                
Operation completed over 1 objects/114.5 MiB.           

In [2]:
! pip install pytorch-pretrained-bert
! wget https://github.com/sugi-chan/custom_bert_pipeline/raw/master/IMDB%20Dataset.csv

     |████████████████████████████████| 133kB 3.4MB/s 
--2020-03-29 04:41:49--  https://github.com/sugi-chan/custom_bert_pipeline/raw/master/IMDB%20Dataset.csv
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sugi-chan/custom_bert_pipeline/master/IMDB%20Dataset.csv [following]
--2020-03-29 04:41:49--  https://raw.githubusercontent.com/sugi-chan/custom_bert_pipeline/master/IMDB%20Dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66212309 (63M) [text/plain]
Saving to: ‘IMDB Dataset.csv’

IMDB Dataset.csv    100%[===================>]  63.14M  69.3MB/s    in 0.9s    

2020-03-29 04:41:50 (69.3 

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.distributed.parallel_loader as pl
import torchvision
from torchvision import datasets, transforms
import copy
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertModel, BertForMaskedLM
import logging
logging.basicConfig(level=logging.INFO)

In [0]:
FLAGS = {}
FLAGS['batch_size'] = 16
FLAGS['num_workers'] = 4
FLAGS['num_epochs'] = 9
FLAGS['log_steps'] = 20
FLAGS['max_seq_length'] = 256
FLAGS['learning_rate'] = 0.01
FLAGS['log_steps'] = 20
FLAGS['metrics_debug'] = False
FLAGS['num_cores'] = 8

In [0]:
config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768, num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)
num_labels = 2

In [0]:
class BertForSequenceClassification(nn.Module):
  def __init__(self, num_labels = 1):
    super(BertForSequenceClassification, self).__init__()
    self.num_labels = num_labels
    self.bert = BertModel.from_pretrained('bert-base-uncased')
    self.dropout = nn.Dropout(config.hidden_dropout_prob)
    self.classifier = nn.Linear(config.hidden_size, num_labels)
    nn.init.xavier_normal_(self.classifier.weight)
  def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
    _,pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
    pooled_output = self.dropout(pooled_output)
    logits = self.classifier(pooled_output)
    return logits

# Custom Data Class Loader
class create_dataset(Dataset):
  def __init__(self, data):
    self.data = data
  def __getitem__(self, index):
    tokenized_text = tokenizer.tokenize(self.data[0][index])
    if len(tokenized_text) > FLAGS['max_seq_length']:
      tokenized_text = tokenized_text[:FLAGS['max_seq_length']]
    ids_text = tokenizer.convert_tokens_to_ids(tokenized_text)
    padding = [0] * (FLAGS['max_seq_length'] - len(tokenized_text))
    ids_text += padding
    assert len(ids_text) == FLAGS['max_seq_length']
    ids_text = torch.tensor(ids_text)
    label = self.data[1][index]
    list_of_labels = [torch.from_numpy(np.array(label))]
    return ids_text, list_of_labels[0]
  def __len__(self):
    return len(self.data[0])


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache, downloading to /tmp/tmp2066v4we
100%|██████████| 231508/231508 [00:00<00:00, 2678735.77B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmp2066v4we to cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmp2066v4we
INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f0006

In [0]:
def train_classifier():
  import pandas as pd
  data = pd.read_csv('IMDB Dataset.csv')
  X = data['review']
  Y = data['sentiment']
  x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.10, random_state=42)
  x_train = x_train.values.tolist()
  x_test = x_test.values.tolist()
  y_train = pd.get_dummies(y_train).values.tolist()
  y_test = pd.get_dummies(y_test).values.tolist()
  train_lists = [x_train, y_train]
  test_lists = [x_test, y_test]
  train_dataset = create_dataset(train_lists)
  test_dataset = create_dataset(test_lists)

  train_sampler = torch.utils.data.DistributedSampler(train_dataset, num_replicas=xm.xrt_world_size(), rank = xm.get_ordinal(), shuffle=True)
  test_sampler = torch.utils.data.DistributedSampler(test_dataset, num_replicas=xm.xrt_world_size(), rank=xm.get_ordinal(), shuffle=False)
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=FLAGS['batch_size'], sampler=train_sampler, num_workers=FLAGS['num_workers'], drop_last=True)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=FLAGS['batch_size'], sampler=test_sampler, shuffle=False, num_workers=FLAGS['num_workers'], drop_last=True)

  data_loaders_dict = {'train': train_loader, 'val': test_loader}
  data_set_sizes = {'train': len(train_lists[0]), 'val': len(test_lists[0])}
  device = xm.xla_device()
  model = BertForSequenceClassification(num_labels).to(device)
  lr_classifier = 0.001
  lr_bert = 0.00001
  optimizer = torch.optim.Adam([
      {"params": model.bert.parameters(), "lr": lr_bert},
      {"params": model.classifier.parameters(), "lr": lr_classifier},
      ])
  criterion = nn.CrossEntropyLoss()
  lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=3, gamma=0.1)

  def train_loop_fn(loader):
    tracker = xm.RateTracker()
    model.train()
    for x, (data, target) in enumerate(loader):
      optimizer.zero_grad()
      outputs = model(data)
      outputs = F.softmax(outputs, dim=1)
      loss = criterion(outputs, torch.max(target.float(), 1)[1])
      loss.backward()
      xm.optimizer_step(optimizer)
      tracker.add(FLAGS['batch_size'])
      if x % FLAGS['log_steps'] == 0:
        print('[xla:{}]({}) Loss={:.5f} Rate={:.2f} GlobalRate={:.2f} Time={}'.format(
              xm.get_ordinal(), x, loss.item(), tracker.rate(),
              tracker.global_rate(), time.asctime()), flush=True)
  def test_loop_fn(loader):
    total_samples =0
    correct = 0
    model.eval()
    data, target = None, None
    for data, target in loader:
      outputs = model(data)
      outputs = F.softmax(outputs, dim=1)
      correct += torch.sum(torch.max(outputs, 1)[1] == torch.max(target, 1)[1])
      total_samples += data.size(0)
    xm.master_print('Sizes: {}, {}'.format(outputs.shape, target.shape))
    accuracy = 100.0 * correct / total_samples
    print('[xla:{}] Accuracy={:.2f}%'.format(
          xm.get_ordinal(), accuracy), flush=True)
    
    return accuracy, model

  for epoch in range(FLAGS['num_epochs']+1):
    para_loader = pl.ParallelLoader(train_loader, [device])
    train_loop_fn(para_loader.per_device_loader(device))
    xm.master_print('Finished Training epoch {}'.format(epoch))

    para_loader = pl.ParallelLoader(test_loader, [device])
    accuracy, model = test_loop_fn(para_loader.per_device_loader(device))
    if FLAGS['metrics_debug']:
        xm.master_print(met.metrics_report(), flush=True)

  return accuracy, model


In [0]:
def _map_fn(rank, flags):
  global FLAGS
  FLAGS = FLAGS
  torch.set_default_tensor_type('torch.FloatTensor')
  accuracy, model = train_classifier()
  xm.save(model.state_dict(), 'checkpoint-1')

In [10]:
xmp.spawn(_map_fn, args=(FLAGS,), nprocs=FLAGS['num_cores'], start_method='fork')

INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz not found in cache, downloading to /tmp/tmp59bq3_m0
100%|██████████| 407873900/407873900 [00:08<00:00, 49999714.46B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmp59bq3_m0 to cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
100%|██████████| 407873900/407873900 [00:13<00:00, 30038298.61B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmpafwyndu0 to cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
100%|██████████| 407873900/407873900 [00:15<00:00, 26269366.16B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmpjfncrdtg to cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fe

[xla:0](0) Loss=0.70844 Rate=1.93 GlobalRate=1.93 Time=Sun Mar 29 04:43:30 2020
[xla:1](0) Loss=0.67006 Rate=1.97 GlobalRate=1.97 Time=Sun Mar 29 04:43:31 2020


INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_att

[xla:7](0) Loss=0.72617 Rate=16.67 GlobalRate=16.67 Time=Sun Mar 29 04:43:44 2020


INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}



[xla:2](0) Loss=0.62905 Rate=15.02 GlobalRate=15.02 Time=Sun Mar 29 04:43:45 2020


INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}



[xla:4](0) Loss=0.74437 Rate=17.29 GlobalRate=17.29 Time=Sun Mar 29 04:43:50 2020
[xla:5](0) Loss=0.68545 Rate=17.50 GlobalRate=17.50 Time=Sun Mar 29 04:43:53 2020
[xla:3](0) Loss=0.72875 Rate=16.31 GlobalRate=16.31 Time=Sun Mar 29 04:43:53 2020
[xla:6](0) Loss=0.62283 Rate=16.33 GlobalRate=16.33 Time=Sun Mar 29 04:43:53 2020
[xla:7](20) Loss=0.61256 Rate=11.57 GlobalRate=8.37 Time=Sun Mar 29 04:44:24 2020
[xla:1](20) Loss=0.62954 Rate=4.41 GlobalRate=5.49 Time=Sun Mar 29 04:44:24 2020
[xla:4](20) Loss=0.62431 Rate=12.57 GlobalRate=9.63 Time=Sun Mar 29 04:44:24 2020
[xla:5](20) Loss=0.65622 Rate=13.22 GlobalRate=10.57 Time=Sun Mar 29 04:44:24 2020
[xla:6](20) Loss=0.69933 Rate=12.79 GlobalRate=10.61 Time=Sun Mar 29 04:44:24 2020
[xla:3](20) Loss=0.75257 Rate=12.77 GlobalRate=10.60 Time=Sun Mar 29 04:44:24 2020
[xla:2](20) Loss=0.59280 Rate=10.94 GlobalRate=8.40 Time=Sun Mar 29 04:44:24 2020
[xla:0](20) Loss=0.69883 Rate=4.39 GlobalRate=5.47 Time=Sun Mar 29 04:44:24 2020
[xla:1](40) Los

In [6]:
model_evaluate = BertForSequenceClassification(2)

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpnvbyvs6g
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}



In [7]:
model_evaluate.load_state_dict(torch.load('checkpoint-1'))

<All keys matched successfully>

In [8]:
model_evaluate

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   